In [ ]:
import pandas as pd
from itertools import chain
from collections import Counter
import re
import numpy as np
import math
from thefuzz import fuzz

In [ ]:
#load data
path_to_dataset  = ""
fb=pd.read_json(path_to_dataset)

In [ ]:
#clean strings (quoted texts and user ids)

fb.quotetext = fb.quotetext.apply (lambda x: [re.sub("\\\\t|\\\\n|\\\\r", "", qq) for qq in x])
fb.quotetext = fb.quotetext.apply (lambda x: [re.sub ('c\\(\\"|\\)', '', qq) for qq in x])
fb.quotetext = fb.quotetext.apply (lambda x: [re.split('.(?=Ursprungligen postat av )', qq) for qq in x])
fb.quotetext = fb.quotetext.apply (lambda x: x[0])
fb.quotetext = fb.quotetext.apply (lambda x: [re.sub('  \\"\\,|  \\"', '', qq) for qq in x])
fb.quotetext = fb.quotetext.apply (lambda x: [re.sub (" +", ' ', qq) for qq in x])
fb.quotetext = fb.quotetext.apply (lambda x: [qq.strip () for qq in x])

fb.user = fb.user.str.lower()
fb.user = fb.user.str.strip()

fb.quote = fb.quote.apply (lambda x: [name.lower() for name in x])
fb.quote = fb.quote.apply (lambda x: [re.sub ("&aring;", "å", quote) for quote in x])
fb.quote = fb.quote.apply (lambda x: [re.sub ("&atilde;", "ã", quote) for quote in x])
fb.quote = fb.quote.apply (lambda x: [re.sub ("&auml;", "ä", quote) for quote in x])
fb.quote = fb.quote.apply (lambda x: [re.sub ("&ouml;", "ö", quote) for quote in x])
fb.quote = fb.quote.apply (lambda x: [name.strip() for name in x])

quotes = fb.quotetext.apply (lambda x: [re.sub ("Ursprungligen postat av \S+.*? ", "", text) for text in x]).tolist()


In [ ]:
#perform fuzzy string mathcing with thefuzz - match initial messages with their qoutes by other users

quoteids = []

for i in range(0, len(quotes)):

    if fb.quoteid[i] != ['']:
        quoteids.append(fb.quoteid[i])  
        continue
    
    elif fb.quote[i] == ['']:
        quoteids.append([''])
        continue
        
    else:
        ids = []
        
        for  j in range(0, len(fb.quote[i])):
            
            df = fb[["postid", "post"]][(fb.user == fb.quote[i][j]) & (fb.header == fb.header[i])].reset_index(drop = True)
            
            if len(df)==0:
                ids.append([np.nan])  
                
            elif len(df)==1:
                ids.append(df.postid[0])    
            
            else:
                rat=[]
                for k in range(0, len(df)):
                    rat.append(fuzz.ratio(quotes[i][j], df.post[k]))
                
                ids.append(df.postid[np.argmax(rat)])

    quoteids.append(ids)

fb['calc_quoteid'] = quoteids

In [ ]:
fb.to_json("flashback_df.json")